<a href="https://colab.research.google.com/github/MJbakhshi/mj/blob/main/mainclassification%20run1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices() #baraye kar kardane gpu


from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Input, Reshape, Lambda
from keras import backend as K
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
from func_utils import *
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ['OMP_NUM_THREADS']='6'
batch_size = 32     #mj #batch_size = 32
epochs = 30         #mj #epochs = 30

In [3]:
print('...loading training data')
f = open('data.pkl', 'rb')
x = pickle.load(f)
f.close()

f = open('data_age.pkl', 'rb')
y = pickle.load(f)
f.close()

f = open('data_gender.pkl','rb')
gender = pickle.load(f)
f.close()

...loading training data


In [4]:
x = np.asarray(x, dtype=np.float32)
y = np.asarray(y)
gender = np.asarray(gender)

In [5]:
x /= 255.
gender =2*( gender-0.5)
x_final = []
y_final = []
gender_final = []

In [6]:
random_no = np.arange(x.shape[0])
np.random.seed(0)
np.random.shuffle(random_no)
for i in random_no:
    x_final.append(x[i,:,:,:])
    y_final.append(y[i])
    gender_final.append(gender[i])

x_final = np.asarray(x_final)
y_final = np.asarray(y_final)
gender_final = np.asarray(gender_final)
print (y_final[:50])
print (gender_final[:50])
k = 500 # Decides split count
x_test = x_final[:k,:,:,:]
y_test = y_final[:k]
gender_test = gender_final[:k]
x_valid = x_final[k:2*k,:,:,:]
y_valid = y_final[k:2*k]
gender_valid = gender_final[k:2*k]
x_train = x_final[2*k:,:,:,:]
y_train = y_final[2*k:]
gender_train = gender_final[2*k:]

[144 126 156 108 228 132 168 138 132 162 144 156 120 132  45  94 120 120
 186  54  78 113 120 162 138 174  60 108 192 138 100 156 150 174 156 156
 168 180  30 120  24  84  60 120 210 120 156 156 189 120]
[-1. -1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1.  1.
  1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1. -1. -1.  1.
  1.  1. -1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1. -1.]


In [7]:
y_train = softlabel(y_train,240)
y_valid = softlabel(y_valid,240)
y_test = softlabel(y_test,240)
print (y_train[0,:])


print ('x_train shape:'+ str(x_train.shape))
print ('y_train shape:'+ str(y_train.shape))
print ('gender_train shape:'+ str(gender_train.shape))
print ('x_valid shape:'+ str(x_valid.shape))
print ('y_valid shape:'+ str(y_valid.shape))
print ('gender_valid shape:' + str(gender_valid.shape))
print ('x_test shape:'+ str(x_test.shape))
print ('y_test shape:'+ str(y_test.shape))

[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.02 0.04 0.06 0.08 0.1
 0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26 0.28 0.3  0.32 0.34 0.36 0.38
 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54 0.56 0.58 0.6  0.62 0.64 0.66
 0.68 0.7  0.72 0.74 0.76 0.78 0.8  0.82 0.84 0.86 0.88 0.9  0.92 0.94
 0.96 0.98 1.   0.98 0.96 0.94 0.92 0.9  0.88 0.86 0.84 0.82 0.8  0.78
 0.76 0.74 0.72 0.7  0.68 0.66 0.64 0.62 0.6  0.58 0.56 0.54 0.52 0.5
 0.48 0.46 0.44 0.42 0.4  0.38 0.36 0.34 0.32 0.3  0.28 0.26 0.24 0.22
 0.2  0.18 0.16 0.14 0

In [8]:
base_model = InceptionV3(weights='imagenet', include_top=False)
for i,layer in enumerate(base_model.layers):
    print (i,layer.name)
input = Input(shape=(300,300,3),name='input1')
input_gender = Input(shape=(1,),dtype='float32',name='input2')
output = base_model(input)
gender_embedding=Dense(16)(input_gender)

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [ ]:
print (K.int_shape(output))
x = keras.layers.MaxPooling2D(pool_size=(8,8))(output)
print (K.int_shape(x))
x=Flatten()(x)
f = keras.layers.Concatenate(axis=1)([x,gender_embedding])
print (K.int_shape(f)) 
#x = Dense(256, activation='relu')(x)
predictions = Dense(240)(x)

model = Model(inputs=[input,input_gender], outputs=predictions)
for i,layer in enumerate(model.layers):
    print (i,layer.name)

Adam=keras.optimizers.Adam(lr=0.0003,beta_1=0.9,beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])

# Save weights after every epoch
checkpoint =keras.callbacks.ModelCheckpoint(filepath='weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5',save_weights_only=True,period=30)
history=model.fit([x_train,gender_train],y_train,batch_size=batch_size,epochs=60,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint]) #mj epochs=60
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

#TestMAE = TestMAE(model,x_test,y_test,gender_test)
#print ('TestMAE:',TestMAE)
##Visulization
weights=model.layers[-1].get_weights()[0]
print (weights.shape)
#GAPAttention(model,weights,'/home/asim/thesis/py36/Bone-Age-Assessment-master/dataset/training/')
#ShowAttentionV1(base_model,'/home/asim/thesis/py36/Bone-Age-Assessment-master/dataset/training/')

#for layer in base_model.layers[:16]:
#    layer.trainable=False
#for layer in base_model.layers:
#    print (layer.name,layer.trainable)
Adam=keras.optimizers.Adam(lr=0.0001,beta_1=0.9,beta_2=0.999)
model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])
history = model.fit([x_train,gender_train],y_train,batch_size=batch_size,epochs=30,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint]) #mj epochs=30
score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test MAE:', score[1])

#TestMAE = TestMAE(model,x_test,y_test,gender_test)
#print ('TestMAE:',TestMAE)

weights=model.layers[-1].get_weights()[0]
print (weights.shape)



/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


(None, 8, 8, 2048)
(None, 1, 1, 2048)
(None, 2064)
0 input1
1 inception_v3
2 max_pooling2d_4
3 flatten
4 input2
5 dense_1
Epoch 1/60
363/363 [==============================] - 55s 99ms/step - loss: 0.1633 - MAE: 0.1633 - val_loss: 0.1151 - val_MAE: 0.1151
Epoch 2/60
363/363 [==============================] - 32s 89ms/step - loss: 0.0929 - MAE: 0.0929 - val_loss: 0.1698 - val_MAE: 0.1698
Epoch 3/60
363/363 [==============================] - 32s 89ms/step - loss: 0.0821 - MAE: 0.0821 - val_loss: 0.0938 - val_MAE: 0.0938
Epoch 4/60
363/363 [==============================] - 32s 89ms/step - loss: 0.0758 - MAE: 0.0758 - val_loss: 3.1053 - val_MAE: 3.1053
Epoch 5/60
363/363 [==============================] - 32s 89ms/step - loss: 0.0730 - MAE: 0.0730 - val_loss: 0.0963 - val_MAE: 0.0963
Epoch 6/60
363/363 [==============================] - 32s 89ms/step - loss: 0.0641 - MAE: 0.0641 - val_loss: 0.1199 - val_MAE: 0.1199
Epoch 7/60
363/363 [==============================] - 32s 89ms/step - loss

In [ ]:

GAPAttention(model,weights,'/home/asim/thesis/py36/Bone-Age-Assessment-master/dataset/training/')
#ShowAttentionV1(base_model,'/home/asim/thesis/py36/Bone-Age-Assessment-master/dataset/training/')

#Adam=keras.optimizers.Adam(lr=0.00001,beta_1=0.9,beta_2=0.999)
#model.compile(optimizer=Adam, loss='mean_absolute_error', metrics=['MAE'])
#history = model.fit([x_train,gender_train], y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=([x_valid,gender_valid],y_valid), callbacks = [checkpoint])
#score = model.evaluate([x_test,gender_test], y_test, batch_size=batch_size)
#print('Test loss:', score[0])
#print('Test MAE:', score[1])

In [ ]:
model.save_weights("model.h5")
with open('history.pkl', 'wb') as f:
	pickle.dump(history.history, f)
f.close()